In [1]:
%%capture
import os
import shutil

contents = os.listdir()

for item in contents:
    if os.path.isfile(item):
        os.remove(item)
        print(f"Удален файл: {item}")
    elif os.path.isdir(item):
        shutil.rmtree(item)
        print(f"Удалена папка: {item}")

In [2]:
%%capture
!pip install lightning

In [3]:
%%capture
! wget https://tudatalib.ulb.tu-darmstadt.de/bitstream/handle/tudatalib/3799/yeast_cell_in_microstructures_dataset.zip
! unzip yeast_cell_in_microstructures_dataset.zip -d yeast_cell_in_microstructures_dataset

In [4]:
from pathlib import Path

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import torch
from matplotlib.figure import Figure
from torch import Tensor, nn
from torch.utils.data import DataLoader, Dataset
from torchmetrics.functional.detection import intersection_over_union
from torchvision.ops.boxes import box_convert

import random
from glob import glob
from pathlib import Path

import lightning as L
import matplotlib.pyplot as plt
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics
import torchvision
from lightning.pytorch.callbacks import Callback
from torch import Tensor
from torchmetrics import Accuracy, Dice, JaccardIndex, Precision, Recall
from torchvision import transforms
from torchvision.utils import make_grid

In [5]:
torch.manual_seed(2004)

## Детекция клеток

Ваша задача: обучить YOLO для детекции дрожжевых клеток и микроструктур (см. [07_object_detection.ipynb](../workshops/07_object_detection.ipynb)). Всё необходимое для запуска обучения вы можете взять из ноутбука с практикой, доделать нужно будет самую малость:
- реализовать расчёт Mean Average Precision для всего валидационного сета
- попробовать привести regression loss к виду, который используется в Yolo9000 и YoloV3
- подобрать лучшие размеры якорных рамок с помощью кластеризации

Основная цель: любыми средствами добиться $mAP > 0.6$ на валидации.

Используйте класс `torchmetrics.detection.MeanAveragePrecision` для расчёта $mAP$.

Нас будет интересовать именно значение `map` в словаре со всеми метриками - это mean average precision, усреднённый по всем отсечкам intersection over union в диапазоне $[0.5, 0.95]$ (см. документацию к классу).

При решении можно пользоваться `lightning` или писать цикл обучения вручную. В последнем случае не забудьте вручную отправить модель и батчи на GPU, чтобы обучалось быстрее

### Задание 1 (3 балла). Цикл обучения с расчётом Mean Average Precision

Запустите обучение модели из практики на всём обучающем датасете, выведите значение $mAP$ на валидационном датасете после окончания обучения.

В этом задании добейтесь $mAP > 0.3$, если всё сделано правильно - для этого должно хватать 30-50 эпох.

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
def process_yolo_preds(preds: Tensor, rescaled_anchors: Tensor) -> tuple[Tensor, Tensor, Tensor]:
    """
    Преобразование выходов модели в
    1. Логит наличия объекта (вероятность получается применением сигмоиды)
    2. Положение рамки относительно ячейки в формате cxcywh
    3. Логиты классов (вероятности получаются применением softmax)
    """
    rescaled_anchors = rescaled_anchors.view(1, len(rescaled_anchors), 1, 1, 2)
    box_predictions = preds[..., 1:5].clone()

    box_predictions[..., 0:2] = torch.sigmoid(box_predictions[..., 0:2])
    box_predictions[..., 2:] = torch.exp(box_predictions[..., 2:]) * rescaled_anchors

    scores = preds[..., 0:1]
    return scores, box_predictions, preds[..., 5:]

In [8]:
GRID_SIZE = 8
IMAGE_SIZE = 256
ANCHORS = [
    [48, 72],
    # [64, 64],
    # [72, 48],
]

rescaled_anchors = torch.tensor(ANCHORS) / IMAGE_SIZE * GRID_SIZE
rescaled_anchors = rescaled_anchors.cuda()

In [9]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)
        self.activation = nn.LeakyReLU(0.1)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return self.activation(x)

class TinyYOLO(nn.Module):
    def __init__(self, num_classes: int = 2, num_anchors: int = 1, in_channels: int = 1) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.in_channels = in_channels
        self.num_anchors = num_anchors
        self.layers = nn.Sequential(
            CNNBlock(1, 16, kernel_size=3, stride=2, padding=1, dilation=2),
            CNNBlock(16, 32, kernel_size=3, stride=2, padding=1, dilation=2),
            CNNBlock(32, 64, kernel_size=3, stride=2, padding=1, dilation=2),
            CNNBlock(64, 128, kernel_size=3, stride=2, padding=1, groups=8),
            CNNBlock(128, 256, kernel_size=3, stride=1, padding=1, groups=8),
            CNNBlock(256, 256, kernel_size=3, stride=1, padding=1, groups=16),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(256, num_anchors * (num_classes + 5), kernel_size=1)
        )

    def forward(self, x: Tensor) -> Tensor:
        x = self.layers(x)
        B, _, W, H = x.shape
        x = x.view(B, self.num_anchors, self.num_classes + 5, W, H)  # B A F W H
        x = x.permute(0, 1, 3, 4, 2)  # B A W H F
        return x


model = TinyYOLO(in_channels=1, num_anchors=1, num_classes=2)
print(sum(p.numel() for p in model.parameters()))
model.forward(torch.randn(2, 1, 256, 256)).shape

109431


torch.Size([2, 1, 8, 8, 7])

In [10]:
def iou_wh(wh1: Tensor, wh2: Tensor) -> Tensor:
    # IoU based on width and height of bboxes

    # intersection
    intersection_area = torch.min(wh1[..., 0], wh2[..., 0]) * torch.min(wh1[..., 1], wh2[..., 1])

    # union
    box1_area = wh1[..., 0] * wh1[..., 1]
    box2_area = wh2[..., 0] * wh2[..., 1]
    union_area = box1_area + box2_area - intersection_area

    iou_score = intersection_area / union_area
    return iou_score


def boxes_to_cells(
    boxes: Tensor,
    classes: Tensor,
    rescaled_anchors: Tensor,
    grid_size: int = 8,
    ignore_iou_thresh: float = 0.5,
) -> Tensor:
    """
    Переводит bbox представление в клеточное представление, где каждая рамка -
    (id класса, вероятность нахождения объекта, cx, cy, w, h), а клеточное представление
    имеет размер (batch_size, n_anchors, grid_size, grid_size, 6), в последней размерности
    хранятся признаки ячейки: класс объекта, вероятность объекта, координаты и размеры рамки
    относительно ячейки

    Args:
        boxes (Tensor): тензор со всеми рамками
        classes (Tensor): тензор с id классов объектов
        rescaled_anchors (Tensor): тензор размера (n_anchors, 2) с размерами якорей в долях от размеров ячейки
        grid_size (int): размер сетки,
        ignore_iou_thresh (float, optional): значение IoU для рамок, при котором ячейка,
            занятая более чем одним объектом, будет специально помечена для игнорирования
    """
    targets = torch.zeros((len(rescaled_anchors), grid_size, grid_size, 6))

    # Каждой рамке сопоставляем клетку и наиболее подходящий якорь
    for box, class_label in zip(boxes, classes):
        iou_anchors = iou_wh(box[2:4], rescaled_anchors / grid_size)
        anchor_indices = iou_anchors.argsort(descending=True, dim=0)
        x, y, width, height = box

        # Относим рамку к наиболее подходящему якорю
        has_anchor = False
        for anchor_idx in anchor_indices:
            s = grid_size

            # Определяем клетку, к которой относится рамка
            i, j = int(s * y), int(s * x)
            anchor_taken = targets[anchor_idx, i, j, 0]

            # Проверяем, доступен ли якорная рамка для текущей ячейки
            if not anchor_taken and not has_anchor:
                # Пересчитываем координаты по отношению к клетке
                x_cell, y_cell = s * x - j, s * y - i
                width_cell, height_cell = (width * s, height * s)
                box_coordinates = torch.tensor([x_cell, y_cell, width_cell, height_cell])

                # Заполняем содержимое для выбранной клетки
                targets[anchor_idx, i, j, 0] = 1  # указатель, что в клетке есть объект
                targets[anchor_idx, i, j, 1:5] = box_coordinates
                targets[anchor_idx, i, j, 5] = int(class_label)

                has_anchor = True

            # Если якорь уже выбран, проверим IoU, если больше threshold - пометим клетку -1
            elif not anchor_taken and iou_anchors[anchor_idx] > ignore_iou_thresh:
                targets[anchor_idx, i, j, 0] = -1

    return targets

In [11]:
class YeastDetectionDataset(Dataset):
    def __init__(
        self, subset_dir: Path, anchors: list[tuple[int, int]], image_size: int, grid_size: int = 8
    ) -> None:
        super().__init__()
        self.subset_dir = subset_dir
        self.items = list((self.subset_dir / "inputs").glob("*.pt"))
        # Ignore IoU threshold
        self.ignore_iou_thresh = 0.5
        self.rescaled_anchors = torch.tensor(anchors) / image_size * grid_size
        self.grid_size = grid_size
        self.image_size = image_size

    def __len__(self) -> int:
        return len(self.items)

    def __getitem__(self, index: int) -> tuple[Tensor, Tensor]:
        image_path = self.items[index]
        # load everything
        image = torch.load(image_path, weights_only=True).unsqueeze(0)
        classes = (
            torch.load(self.subset_dir / "classes" / image_path.parts[-1], weights_only=True) + 1
        )
        boxes = torch.load(
            self.subset_dir / "bounding_boxes" / image_path.parts[-1], weights_only=True
        )
        boxes = box_convert(boxes, "xyxy", "cxcywh") / self.image_size

        # convert boxes to cells
        targets = boxes_to_cells(
            boxes, classes, self.rescaled_anchors, self.grid_size, self.ignore_iou_thresh
        )
        return image, targets

In [12]:
train_dataset = YeastDetectionDataset(
    Path("yeast_cell_in_microstructures_dataset/train"), anchors=ANCHORS, image_size=256
)
val_dataset = YeastDetectionDataset(Path("yeast_cell_in_microstructures_dataset/val"), anchors=ANCHORS, image_size=256)

batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [13]:
def cells_to_bboxes(cells: Tensor, rescaled_anchors: Tensor, is_predictions=False) -> Tensor:
    """
    Переводит клеточное представление в bbox представление, где каждая рамка -
    (id класса, вероятность нахождения объекта, cx, cy, w, h), а клеточное представление
    имеет размер (batch_size, n_anchors, grid_size, grid_size, 6), в последней размерности
    хранятся признаки ячейки: класс объекта, вероятность объекта, координаты и размеры рамки
    относительно ячейки

    Args:
        cells (Tensor): тензор размера (batch_size, n_anchors, width, height, 6)
        rescaled_anchors (Tensor): тензор размера (n_anchors, 2) с размерами якорей в долях от размеров ячейки
        is_predictions (bool, optional): являются ли входные ячейки предсказаниями или верной аннотацией.
    """

    if is_predictions:
        scores, box_predictions, logits = process_yolo_preds(cells, rescaled_anchors)
        scores = torch.sigmoid(scores)
        best_class = torch.argmax(logits, dim=-1).unsqueeze(-1) + 1

    else:
        box_predictions = cells[..., 1:5].clone()
        scores = cells[..., 0:1]
        best_class = cells[..., 5:6]

    # масштабируем размер рамок [0, grid_size] -> [0, 1]
    _, _, H, W, _ = cells.shape
    range_y, range_x = torch.meshgrid(
        torch.arange(H, dtype=cells.dtype, device=cells.device),
        torch.arange(W, dtype=cells.dtype, device=cells.device),
        indexing="ij",
    )
    x = torch.cat(
        [
            best_class,
            scores,
            (box_predictions[..., 0:1] + range_x[None, None, :, :, None]) / W,  # X center
            (box_predictions[..., 1:2] + range_y[None, None, :, :, None]) / H,  # Y center
            box_predictions[..., 2:3] / W,  # Width
            box_predictions[..., 3:4] / H,  # Height
        ],
        -1,
    )

    return x.view(-1, 6)

In [14]:
class YOLOLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        self.bce = nn.BCEWithLogitsLoss()
        self.cross_entropy = nn.CrossEntropyLoss()

    def forward(self, pred: Tensor, target: Tensor, anchors: Tensor) -> Tensor:
        anchors = anchors.to(pred.device)
        # ниже входные тензоры будут меняться на месте, так что склонируем A
        pred = pred.clone()
        target = target.clone()

        # разделяем рамки на содержащие объекты и не содержащие
        # NB: ещё могут быть -1, куда отнеслось более 1 объекта - их не учитываем
        obj = target[..., 0] == 1
        no_obj = target[..., 0] == 0

        # преобразуем предсказания bbox
        scores, pred_boxes, logits = process_yolo_preds(pred, anchors)

        # no object loss: кросс-энтропия вместо MSE
        no_object_loss = self.bce(
            (scores[no_obj]),
            (target[..., 0:1][no_obj]),
        )

        # object loss: учим предсказывать IoU
        ious = intersection_over_union(pred_boxes[obj], target[..., 1:5][obj]).detach()
        object_loss = self.mse(scores[obj].sigmoid(), ious * target[..., 0:1][obj])

        # box coordinate loss: логарифмируем размеры рамок перед расчётом MSE
        anchors = anchors.reshape(1, len(anchors), 1, 1, 2)
        pred[..., 1:3] = pred[..., 1:3].sigmoid()
        target[..., 3:5] = torch.log(1e-6 + target[..., 3:5] / anchors)
        box_loss = self.mse(pred[..., 1:5][obj], target[..., 1:5][obj])

        # class loss: здесь всё обычно
        class_loss = self.cross_entropy(logits[obj], target[..., 5][obj].long() - 1)

        # Total loss
        return box_loss + object_loss + no_object_loss + class_loss

In [15]:
import pytorch_lightning as L
import torch
from torch import nn, optim

class YOLOModule(L.LightningModule):
    def __init__(self, model, loss_fn, learning_rate=1e-3):
        super().__init__()
        self.model = model
        self.loss_fn = loss_fn
        self.learning_rate = learning_rate
        self.map_metric = MeanAveragePrecision(box_format='cxcywh', iou_type='bbox', class_metrics=True)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, targets = batch
        images = images.cuda()
        targets = targets.cuda()
        predictions = self(images)
        loss = self.loss_fn(predictions, targets, rescaled_anchors.to(self.device))
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, targets = batch
        images = images.cuda()
        targets = targets.cuda()
        predictions = self(images)
        loss = self.loss_fn(predictions, targets, rescaled_anchors.cuda())
        self.log('val_loss', loss, prog_bar=True)

        pred_boxes = cells_to_bboxes(predictions, rescaled_anchors.to(self.device), is_predictions=True)
        target_boxes = cells_to_bboxes(targets, rescaled_anchors.to(self.device))

        preds = [{'boxes': pred_boxes[:, 2:], 'scores': pred_boxes[:, 1], 'labels': pred_boxes[:, 0].long()}]
        targets = [{'boxes': target_boxes[:, 2:], 'labels': target_boxes[:, 0].long()}]

        self.map_metric.update(preds, targets)

    def on_validation_epoch_end(self):
        map_result = self.map_metric.compute()
        self.log('val_mAP', map_result['map'], prog_bar=True)
        self.map_metric.reset()

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [16]:
model = TinyYOLO(in_channels=1, num_anchors=len(rescaled_anchors), num_classes=2)
loss_fn = YOLOLoss()

yolo_module = YOLOModule(model, loss_fn, learning_rate=1e-3)
yolo_module = yolo_module.cuda()

trainer = L.Trainer(
    max_epochs=60,
    accelerator='gpu',
    devices=1,
    log_every_n_steps=10,
    callbacks=[L.callbacks.ModelCheckpoint(monitor='val_mAP', mode='max')]
)

trainer.fit(yolo_module, train_loader, val_loader)

final_map = trainer.callback_metrics['val_mAP'].item()
print(f"Final mAP: {final_map:.4f}")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                 | Params | Mode 
------------------------------------------------------------
0 | model      | TinyYOLO             | 109 K  | train
1 | loss_fn    | YOLOLoss             | 0      | train
2 | map_metric | MeanAveragePrecision | 0      | train
------------------------------------------------------------
109 K     Trainable params
0         Non-trainable params
109 K     Total params
0.438     Total estimated model params size (MB)
33        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)  # noqa: B028


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=60` reached.


Final mAP: 0.3183


### Задание 2 (2 балла). YoloV3 loss

Мы упоминали, что на практике использовалась не совсем та же самая ошибка, что и в YOLO. В этом задании исправьте в классе YoloLoss ошибку регрессии, приведя её в соответствие с тем, как она описана в статье [YOLOv3: An Incremental Improvement](https://arxiv.org/abs/1804.02767) (см. раздел 2.1. Bounding Box Prediction).

Запустите обучение с изменённой ошибкой, добейтесь $mAP > 0.3$

In [17]:
def process_yolo_preds(preds: Tensor, rescaled_anchors: Tensor) -> tuple[Tensor, Tensor, Tensor]:
    """
    Преобразование выходов модели в
    1. Логит наличия объекта (вероятность получается применением сигмоиды)
    2. Положение рамки относительно ячейки в формате cxcywh
    3. Логиты классов (вероятности получаются применением softmax)
    """
    rescaled_anchors = rescaled_anchors.view(1, len(rescaled_anchors), 1, 1, 2)
    box_predictions = preds[..., 1:5].clone()

    box_predictions[..., 0:2] = torch.sigmoid(box_predictions[..., 0:2])
    box_predictions[..., 2:] = torch.exp(box_predictions[..., 2:]) * rescaled_anchors

    scores = preds[..., 0:1]
    return scores, box_predictions, preds[..., 5:]

In [18]:
class YOLOLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        self.bce = nn.BCEWithLogitsLoss()
        self.cross_entropy = nn.CrossEntropyLoss()

    def forward(self, pred: Tensor, target: Tensor, anchors: Tensor) -> Tensor:
        anchors = anchors.to(pred.device)
        # ниже входные тензоры будут меняться на месте, так что склонируем A
        pred = pred.clone()
        target = target.clone()

        # разделяем рамки на содержащие объекты и не содержащие
        # NB: ещё могут быть -1, куда отнеслось более 1 объекта - их не учитываем
        obj = target[..., 0] == 1
        no_obj = target[..., 0] == 0

        # преобразуем предсказания bbox
        scores, pred_boxes, logits = process_yolo_preds(pred, anchors)

        # no object loss: кросс-энтропия вместо MSE
        no_object_loss = self.bce(
            (scores[no_obj]),
            (target[..., 0:1][no_obj]),
        )

        # object loss: учим предсказывать IoU
        ious = intersection_over_union(pred_boxes[obj], target[..., 1:5][obj]).detach()
        object_loss = self.mse(scores[obj].sigmoid(), ious * target[..., 0:1][obj])

        # box coordinate loss: логарифмируем размеры рамок перед расчётом MSE
        anchors = anchors.reshape(1, len(anchors), 1, 1, 2)
        pred[..., 1:3] = pred[..., 1:3].sigmoid()
        target[..., 3:5] = torch.log(1e-6 + target[..., 3:5] / anchors)
        box_loss = self.mse(pred[..., 1:5][obj], target[..., 1:5][obj])

        # class loss: здесь всё обычно
        class_loss = self.cross_entropy(logits[obj], target[..., 5][obj].long() - 1)

        # Total loss
        return box_loss + object_loss + no_object_loss + class_loss

In [19]:
class YOLOLoss_orig(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        self.bce = nn.BCEWithLogitsLoss()
        self.cross_entropy = nn.CrossEntropyLoss()

    def forward(self, pred: Tensor, target: Tensor, anchors: Tensor) -> Tensor:
        anchors = anchors.to(pred.device)
        # ниже входные тензоры будут меняться на месте, так что склонируем их
        pred = pred.clone()
        target = target.clone()

        # разделяем рамки на содержащие объекты и не содержащие
        # NB: ещё могут быть -1, куда отнеслось более 1 объекта - их не учитываем
        obj = target[..., 0] == 1
        no_obj = target[..., 0] == 0

        # преобразуем предсказания bbox
        scores, pred_boxes, logits = process_yolo_preds(pred, anchors)

        # no object loss: кросс-энтропия вместо MSE
        no_object_loss = self.bce(
            (scores[no_obj]),
            (target[..., 0:1][no_obj]),
        )

        # object loss: учим предсказывать IoU
        ious = intersection_over_union(pred_boxes[obj], target[..., 1:5][obj]).detach()
        object_loss = self.mse(scores[obj].sigmoid(), ious * target[..., 0:1][obj])

        # box coordinate loss: логарифмируем размеры рамок перед расчётом MSE
        grid_size = pred.shape[2:4]

        # grid_x = torch.arange(grid_size[1]).view(1, 1, 1, grid_size[1]).expand(1, 1, grid_size[0], grid_size[1])
        # grid_y = torch.arange(grid_size[0]).view(1, 1, grid_size[0], 1).expand(1, 1, grid_size[0], grid_size[1])

        # grid_x = torch.arange(grid_size).repeat(grid_size, 1).view([1, 1, grid_size, grid_size]).float()
        # grid_y = torch.arange(grid_size).repeat(grid_size, 1).t().view([1, 1, grid_size, grid_size]).float()

        grid_y, grid_x = torch.meshgrid([torch.arange(grid_size[0]), torch.arange(grid_size[1])])

        grid_xy = torch.stack((grid_x, grid_y), dim=-1).to(pred.device).unsqueeze(0).unsqueeze(0)

        anchors = anchors.reshape(1, len(anchors), 1, 1, 2)
        pred[..., 1:3] = pred[..., 1:3].sigmoid() + grid_xy
        # pred[..., 3:5] = torch.exp(pred[..., 3:5]) * anchors

        target[..., 3:5] = torch.log(1e-6 + target[..., 3:5] / anchors)
        box_loss = self.mse(pred[..., 1:5][obj], target[..., 1:5][obj])

        # class loss: здесь всё обычно
        class_loss = self.cross_entropy(logits[obj], target[..., 5][obj].long() - 1)

        # Total loss
        return 10 * box_loss + object_loss + 10 * no_object_loss + class_loss

In [20]:
GRID_SIZE = 8
IMAGE_SIZE = 256
ANCHORS = [
    [48, 72],
    [64, 64],
    [72, 48],
]

ANCHORS = [
    [57.31111, 87.74074],
    [42.71875, 44.21875],
    [54.79661, 61.398304],
    [75.49152, 68.66102],
    [71.98203, 104.97205],
]

rescaled_anchors = torch.tensor(ANCHORS) / IMAGE_SIZE * GRID_SIZE
rescaled_anchors = rescaled_anchors.cuda()

In [21]:
train_dataset = YeastDetectionDataset(
    Path("yeast_cell_in_microstructures_dataset/train"), anchors=ANCHORS, image_size=256
)
val_dataset = YeastDetectionDataset(Path("yeast_cell_in_microstructures_dataset/val"), anchors=ANCHORS, image_size=256)

batch_size = 32
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [22]:
model = TinyYOLO(in_channels=1, num_anchors=len(rescaled_anchors), num_classes=2)
loss_fn = YOLOLoss_orig()

yolo_module = YOLOModule(model, loss_fn, learning_rate=1e-3)
yolo_module = yolo_module.cuda()

trainer = L.Trainer(
    max_epochs=60,
    accelerator='gpu',
    devices=1,
    log_every_n_steps=10,
    callbacks=[L.callbacks.ModelCheckpoint(monitor='val_mAP', mode='max')]
)

trainer.fit(yolo_module, train_loader, val_loader)

final_map = trainer.callback_metrics['val_mAP'].item()
print(f"Final mAP: {final_map:.4f}")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                 | Params | Mode 
------------------------------------------------------------
0 | model      | TinyYOLO             | 116 K  | train
1 | loss_fn    | YOLOLoss_orig        | 0      | train
2 | map_metric | MeanAveragePrecision | 0      | train
------------------------------------------------------------
116 K     Trainable params
0         Non-trainable params
116 K     Total params
0.467     Total estimated model params size (MB)
33        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=60` reached.


Final mAP: 0.0260


### Задание 3 (3 балла). Выбор anchors с помощью кластеризации

В статье [YOLO9000: Better, Faster, Stronger](https://arxiv.org/abs/1612.08242) в разделе 2. Better. Dimension clusters описан способ выбора anchor boxes через кластеризацию обучающего датасета.

Проделайте то же самое с вашим обучающим датасетом, чтобы выбрать несколько anchor boxes.

В качестве результата выведите получившиеся размеры anchors для # Clusters = 5

In [23]:
import numpy as np
import torch
from pathlib import Path
from torchvision.ops import box_convert

def load_boxes(dataset_path):
    boxes = []
    for image_path in Path(dataset_path).glob("*.pt"):
        box = torch.load(Path("yeast_cell_in_microstructures_dataset/train/bounding_boxes") / image_path.name, weights_only=True)
        boxes.extend([box_convert(b, "xyxy", "cxcywh")[2:] for b in box])
    return boxes

def kmeans_clustering(boxes, k, max_iterations=100):
    centroids = np.array(boxes)[np.random.choice(len(boxes), k, replace=False)]

    for _ in range(max_iterations):
        clusters = [[] for _ in range(k)]

        for box in boxes:
            distances = [1 - iou_wh(box, torch.tensor(centroid)) for centroid in centroids]
            closest_centroid = np.argmin(distances)
            clusters[closest_centroid].append(box)

        new_centroids = [np.mean(cluster, axis=0) if cluster else centroid
                         for cluster, centroid in zip(clusters, centroids)]

        if np.allclose(centroids, new_centroids):
            break

        centroids = new_centroids

    return np.array(centroids)

In [24]:
dataset_path = "yeast_cell_in_microstructures_dataset/train/inputs"

boxes = load_boxes(dataset_path)
centroids = kmeans_clustering(boxes, k=5)

print(centroids)

[[ 70.28755  105.56223 ]
 [ 57.17178   84.828224]
 [ 43.632183  48.04598 ]
 [ 91.29851   88.31343 ]
 [ 61.798386  62.064518]]


### Задание 4 (4 балла + бонусы за лучшую точность). Обучите модель


Ваша цель: $mAP > 0.6$ на валидации.

Можете использовать весь арсенал:
- использование множества якорных рамок (сформированных вручную или в результате кластеризации)
- любые изменения функции ошибки
- любые изменения архитектуры модели и регуляризация
- аугментации (вспоминаем `torchvision.transforms` и `albumentations`)
- любая длительность обучения, оптимизатор, расписание для learning rate

Бонусы за повышенную точность:
- **5 баллов**: $mAP > 0.65$
- **1 балл** за каждые следующие $0.01$ (т. е. за $mAP > 0.72$ в этом задании вы получите $4 + 12 = 16$ баллов)

**Важно**: перез запуском обучения зафиксируйте `torch.manual_seed()`

In [25]:
class CNNBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)
        self.activation = nn.LeakyReLU(0.1)

    def forward(self, x: Tensor) -> Tensor:
        x = self.conv(x)
        x = self.bn(x)
        return self.activation(x)


class ResidualBlock(nn.Module):
    def __init__(self, inplanes: int, planes: int) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(
            inplanes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(
            planes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Conv2d(
            inplanes, planes, kernel_size=1, stride=1, bias=False
        )
        self.bn_shortcut = nn.BatchNorm2d(planes)

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        identity = self.shortcut(identity)
        identity = self.bn_shortcut(identity)

        out += identity
        out = self.relu(out)

        return out

class DeepYOLO(nn.Module):
    def __init__(self, num_classes: int = 2, num_anchors: int = 1, in_channels: int = 1) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.num_anchors = num_anchors

        self.layers = nn.Sequential(
            CNNBlock(in_channels, 32, kernel_size=3, stride=2, padding=1),
            CNNBlock(32, 64, kernel_size=3, stride=2, padding=1),
            ResidualBlock(64, 64),
            CNNBlock(64, 128, kernel_size=3, stride=2, padding=1),
            ResidualBlock(128, 128),
            CNNBlock(128, 256, kernel_size=3, stride=2, padding=1),
            ResidualBlock(256, 256),
            CNNBlock(256, 512, kernel_size=3, stride=2, padding=1),
            ResidualBlock(512, 512),
            CNNBlock(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(1024, num_anchors * (num_classes + 5), kernel_size=1)  # Финальный слой
        )

    def forward(self, x: Tensor) -> Tensor:
        x = self.layers(x)
        B, _, W, H = x.shape
        x = x.view(B, self.num_anchors, self.num_classes + 5, W, H)  # B A F W H
        x = x.permute(0, 1, 3, 4, 2)  # B A W H F
        return x

Число параметров: 12917607
Выходной shape: torch.Size([2, 1, 8, 8, 7])


In [26]:
GRID_SIZE = 8
IMAGE_SIZE = 256
ANCHORS = [
    [48, 72],
    [64, 64],
    [72, 48],
]

ANCHORS = [
    [57.31111, 87.74074],
    [42.71875, 44.21875],
    [54.79661, 61.398304],
    [75.49152, 68.66102],
    [71.98203, 104.97205],
]

rescaled_anchors = torch.tensor(ANCHORS) / IMAGE_SIZE * GRID_SIZE
rescaled_anchors = rescaled_anchors.cuda()

In [28]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

train_transforms = A.Compose([
    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.5),
    A.GaussianBlur(blur_limit=(3, 5), p=0.3),
    ToTensorV2(),
])

val_transforms = A.Compose([
    ToTensorV2(),
])

In [29]:
class YeastDetectionDataset(Dataset):
    def __init__(
        self,
        subset_dir: Path,
        anchors: list[tuple[int, int]],
        image_size: int,
        grid_size: int = 8,
        transform=None
    ) -> None:
        super().__init__()
        self.subset_dir = subset_dir
        self.items = list((self.subset_dir / "inputs").glob("*.pt"))
        self.ignore_iou_thresh = 0.5
        self.rescaled_anchors = torch.tensor(anchors) / image_size * grid_size
        self.grid_size = grid_size
        self.image_size = image_size
        self.transform = transform

    def __len__(self) -> int:
        return len(self.items)

    def __getitem__(self, index: int) -> tuple[Tensor, Tensor]:
        image_path = self.items[index]
        image = torch.load(image_path, weights_only=True).unsqueeze(0)
        classes = (
            torch.load(self.subset_dir / "classes" / image_path.parts[-1], weights_only=True) + 1
        )
        boxes = torch.load(
            self.subset_dir / "bounding_boxes" / image_path.parts[-1], weights_only=True
        )
        boxes = box_convert(boxes, "xyxy", "cxcywh") / self.image_size

        if self.transform:
            image_np = image.squeeze(0).numpy()
            transformed = self.transform(image=image_np)
            image = transformed['image'].unsqueeze(0)

        image = image.squeeze(1)

        targets = boxes_to_cells(
            boxes, classes, self.rescaled_anchors, self.grid_size, self.ignore_iou_thresh
        )
        return image, targets


In [30]:
train_dataset = YeastDetectionDataset(
    Path("yeast_cell_in_microstructures_dataset/train"),
    anchors=ANCHORS,
    image_size=IMAGE_SIZE,
    # transform=None,
    transform=train_transforms
)

val_dataset = YeastDetectionDataset(
    Path("yeast_cell_in_microstructures_dataset/val"),
    anchors=ANCHORS,
    image_size=IMAGE_SIZE,
    # transform=None,
    transform=val_transforms
)

batch_size = 16

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)


In [31]:
import pytorch_lightning as L
import torch
from torch import nn, optim
from typing import Any

class YOLOModule(L.LightningModule):
    def __init__(self, model, loss_fn, learning_rate=1e-3):
        super().__init__()
        self.model = model
        self.loss_fn = loss_fn
        self.learning_rate = learning_rate
        self.map_metric = MeanAveragePrecision(box_format='cxcywh', iou_type='bbox', class_metrics=True)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, targets = batch
        images = images.cuda()
        targets = targets.cuda()
        predictions = self(images)
        loss = self.loss_fn(predictions, targets, rescaled_anchors.to(self.device))
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, targets = batch
        images = images.cuda()
        targets = targets.cuda()
        predictions = self(images)
        loss = self.loss_fn(predictions, targets, rescaled_anchors.cuda())
        self.log('val_loss', loss, prog_bar=True)

        pred_boxes = cells_to_bboxes(predictions, rescaled_anchors.to(self.device), is_predictions=True)
        target_boxes = cells_to_bboxes(targets, rescaled_anchors.to(self.device))

        preds = [{'boxes': pred_boxes[:, 2:], 'scores': pred_boxes[:, 1], 'labels': pred_boxes[:, 0].long()}]
        targets = [{'boxes': target_boxes[:, 2:], 'labels': target_boxes[:, 0].long()}]

        self.map_metric.update(preds, targets)

    def on_validation_epoch_end(self):
        map_result = self.map_metric.compute()
        self.log('val_mAP', map_result['map'], prog_bar=True)
        self.map_metric.reset()

    def configure_optimizers(self) -> dict[str, Any]:
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate, weight_decay=1e-5)
        return {
            "optimizer": optimizer,
            "lr_scheduler": torch.optim.lr_scheduler.MultiStepLR(
                optimizer, milestones=[35, 40], gamma=0.25
            ),
        }
        return optimizer

In [32]:
model = DeepYOLO(in_channels=1, num_anchors=len(rescaled_anchors), num_classes=2)
loss_fn = YOLOLoss()

yolo_module = YOLOModule(model, loss_fn, learning_rate=1e-3)
yolo_module = yolo_module.cuda()

trainer = L.Trainer(
    max_epochs=100,
    accelerator='gpu',
    devices=1,
    log_every_n_steps=10,
    callbacks=[L.callbacks.ModelCheckpoint(monitor='val_mAP', mode='max')]
)

trainer.fit(yolo_module, train_loader, val_loader)

final_map = trainer.callback_metrics['val_mAP'].item()
print(f"Final mAP: {final_map:.4f}")

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                 | Params | Mode 
------------------------------------------------------------
0 | model      | DeepYOLO             | 12.9 M | train
1 | loss_fn    | YOLOLoss             | 0      | train
2 | map_metric | MeanAveragePrecision | 0      | train
------------------------------------------------------------
12.9 M    Trainable params
0         Non-trainable params
12.9 M    Total params
51.785    Total estimated model params size (MB)
64        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:43: UserWarning: Encountered more than 100 detections in a single image. This means that certain detections with the lowest scores will be ignored, that may have an undesirable impact on performance. Please consider adjusting the `max_detection_threshold` to suit your use case. To disable this warning, set attribute class `warn_on_many_detections=False`, after initializing the metric.
  warnings.warn(*args, **kwargs)  # noqa: B028


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


Final mAP: 0.4681


### Efficientnet

In [33]:
from torchvision.models import efficientnet_b5, EfficientNet_B5_Weights
from torchvision.models import efficientnet_b6, EfficientNet_B6_Weights

In [34]:
class DeepYOLO_Efnet(nn.Module):
    def __init__(self, num_classes: int = 2, num_anchors: int = 1, in_channels: int = 1) -> None:
        super().__init__()
        self.num_classes = num_classes
        self.num_anchors = num_anchors

        self.channel_expansion = nn.Conv2d(in_channels, 3, kernel_size=1)

        self.backbone = efficientnet_b5(weights=EfficientNet_B5_Weights.DEFAULT).features

        out_channels = 2048

        self.additional_layers = nn.Sequential(
            CNNBlock(out_channels, 1024, kernel_size=3, stride=1, padding=1),
            ResidualBlock(1024, 1024),
            CNNBlock(1024, 512, kernel_size=3, stride=1, padding=1),
            ResidualBlock(512, 512),
            CNNBlock(512, 1024, kernel_size=3, stride=1, padding=1),
            nn.Conv2d(1024, num_anchors * (num_classes + 5), kernel_size=1)
        )

    def forward(self, x: Tensor) -> Tensor:
        x = self.channel_expansion(x)

        x = self.backbone(x)
        x = self.additional_layers(x)

        B, _, W, H = x.shape
        x = x.view(B, self.num_anchors, self.num_classes + 5, W, H)
        x = x.permute(0, 1, 3, 4, 2)
        return x

In [35]:
GRID_SIZE = 8
IMAGE_SIZE = 256
ANCHORS = [
    [48, 72],
    [64, 64],
    [72, 48],
]

ANCHORS = [
    [57.31111, 87.74074],
    [42.71875, 44.21875],
    [54.79661, 61.398304],
    [75.49152, 68.66102],
    [71.98203, 104.97205],
]

rescaled_anchors = torch.tensor(ANCHORS) / IMAGE_SIZE * GRID_SIZE
rescaled_anchors = rescaled_anchors.cuda()

In [36]:
train_dataset = YeastDetectionDataset(
    Path("yeast_cell_in_microstructures_dataset/train"),
    anchors=ANCHORS,
    image_size=IMAGE_SIZE,
    transform=None,
    # transform=train_transforms
)

val_dataset = YeastDetectionDataset(
    Path("yeast_cell_in_microstructures_dataset/val"),
    anchors=ANCHORS,
    image_size=IMAGE_SIZE,
    transform=None,
    # transform=val_transforms
)

batch_size = 16

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)


In [37]:
import pytorch_lightning as L
import torch
from torch import nn, optim
from torchmetrics.detection.mean_ap import MeanAveragePrecision

class YOLOModule(L.LightningModule):
    def __init__(self, model, loss_fn, learning_rate=1e-3):
        super().__init__()
        self.model = model
        self.loss_fn = loss_fn
        self.learning_rate = learning_rate
        self.map_metric = MeanAveragePrecision(box_format='cxcywh', iou_type='bbox', class_metrics=True)

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, targets = batch
        images = images.cuda()
        targets = targets.cuda()
        predictions = self(images)
        loss = self.loss_fn(predictions, targets, rescaled_anchors.to(self.device))
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, targets = batch
        images = images.cuda()
        targets = targets.cuda()
        predictions = self(images)
        loss = self.loss_fn(predictions, targets, rescaled_anchors.cuda())
        self.log('val_loss', loss, prog_bar=True)

        pred_boxes = cells_to_bboxes(predictions, rescaled_anchors.to(self.device), is_predictions=True)
        target_boxes = cells_to_bboxes(targets, rescaled_anchors.to(self.device))

        preds = [{'boxes': pred_boxes[:, 2:], 'scores': pred_boxes[:, 1], 'labels': pred_boxes[:, 0].long()}]
        targets = [{'boxes': target_boxes[:, 2:], 'labels': target_boxes[:, 0].long()}]

        self.map_metric.update(preds, targets)

    def on_validation_epoch_end(self):
        map_result = self.map_metric.compute()
        self.log('val_mAP', map_result['map'], prog_bar=True)
        self.map_metric.reset()

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [38]:
model = DeepYOLO_Efnet(in_channels=1, num_anchors=len(rescaled_anchors), num_classes=2)
loss_fn = YOLOLoss()

yolo_module = YOLOModule(model, loss_fn, learning_rate=1e-3)
yolo_module = yolo_module.cuda()

trainer = L.Trainer(
    max_epochs=100,
    accelerator='gpu',
    devices=1,
    log_every_n_steps=10,
    callbacks=[L.callbacks.ModelCheckpoint(monitor='val_mAP', mode='max')]
)

trainer.fit(yolo_module, train_loader, val_loader)

final_map = trainer.callback_metrics['val_mAP'].item()
print(f"Final mAP: {final_map:.4f}")

Downloading: "https://download.pytorch.org/models/efficientnet_b5_lukemelas-1a07897c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b5_lukemelas-1a07897c.pth
100%|██████████| 117M/117M [00:01<00:00, 87.5MB/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type                 | Params | Mode 
------------------------------------------------------------
0 | model      | DeepYOLO_Efnet       | 81.6 M | train
1 | loss_fn    | YOLOLoss             | 0      | train
2 | map_metric | MeanAveragePrecision | 0      | train
------------------------------------------------------------
81.6 M    Trainable params
0         Non-trainable params
81.6 M

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.


Final mAP: 0.4332
